# DAIN Colab alpha

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

My fork:
https://github.com/styler00dollar/DAIN

Enhancement by Styler00Dollar aka "sudo rm -rf / --no-preserve-root#8353" on discord. Please do not run this command in your linux terminal. It's rather meant as a joke.

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- If you want to interpolate a png-sequence, put your files with xxxxx.png starting at 1 in content/DAIN/MiddleBurySet/other-data/Beanbags/. If the input is a video, then ffmpeg will automatically use that folder.
- Don't forget to configure the filename in Colab. You will notice that down below.
- The default configuration is 4x. Change the code yourself if you want something different. This alpha does not include an easy option.
- I DO NOT recommend to "run all". Run a cell and wait, then run the next cell.


Stuff that should be improved:
- Currently the output does have some random limitations due to my code. No more than 100 Frames will be processed. Output also does have a 4-digit limit. Don't try something too big. This is an alpha that should work on small stuff. I should fix that later.
- Gif (or RGBA24?) seems to be broken. A mkv-video file worked fine.
- A filename variable.
- People that look at my Code will notice that I use a Folder named "Beanbags". Why?? I don't know.
- Maybe temp-copy to .ipynb_checkpoints can be avoided.
- Changing ffmpeg ouptut commands, so new framerate will be inserted automatically.
- Handling png-sequence and video better. PNG should be able to be copied from drive directly into the correct folder.
- Adding menu to select speed.

In [0]:
################# Configuration cell! ############################
# Use the values in here to configure what you'd like DAIN to do.

# Input file: Path (relative to the root of your Google Drive) to the input file.
# For instance, if you save your "example.mkv" file in your Google Drive, inside a "videos" folder, the path would be:
# videos/example.mkv
INPUT_FILEPATH = "DAIN/original.mp4"

# Target FPS = how many frames per second should the result have. This will determine how many intermediate images are
# interpolated.
TARGET_FPS = 60

# Output file path: path (relative to the root of your Google Drive) for the output file.
# Extension should always be MKV.
OUTPUT_FILE_PATH = "DAIN/output.mkv"

In [3]:
# Connect Google Drive

from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/DAIN/'
print('Google Drive connected.')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Google Drive connected.


In [0]:
# Standard ffmpeg
#!apt install ffmpeg

In [4]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# P100: 16GB (Can handle 720p. 1080p does not work and you will get an out-of-memory error.)
# P4: 8GB (Not tested.)

#%cd /tmp
#!youtube-dl -f 299 --postprocessor-args "-ss 0:0:1 -to 0:0:20" "https://www.youtube.com/watch?v=hU_NLZYUDyI"
#!nvidia-smi --query-gpu=gpu_name --format=csv
#!wget -O "gdrive/My Drive/ESRGANModels/dejpeg1.pth" https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader

Tesla P4


In [4]:
!pip install opencv-python
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from IPython.display import clear_output

In [8]:
!rm -rf /content/DAIN
!git clone https://github.com/AlphaGit/DAIN.git

Cloning into 'DAIN'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 611 (delta 4), reused 3 (delta 1), pack-reused 599
Receiving objects: 100% (611/611), 235.31 KiB | 14.71 MiB/s, done.
Resolving deltas: 100% (338/338), done.


In [17]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
clear_output()
print("Building done.")

Building done.


In [19]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
clear_output()
print("Building done.")

Building done.


In [0]:
# Downloading pre-trained models
%cd /content/DAIN
!mkdir model_weights MiddleBurySet
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

%cd /content/DAIN/MiddleBurySet
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip

!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip

clear_output()

In [0]:
!CUDA_VISIBLE_DEVICES=0

In [23]:
# Fix scipy
# DO NOT RESTART YOUR COLAB!! Yes, there is red text and a warning, but it works.

#!pip uninstall scipy
!pip install --force-reinstall scipy==1.0.0

     |████████████████████████████████| 50.0MB 58kB/s 
     |████████████████████████████████| 20.2MB 155kB/s 
ERROR: seaborn 0.10.0 has requirement scipy>=1.0.1, but you'll have scipy 1.0.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.25 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [56]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"FPS: {fps}")

FPS: 10.0


In [57]:
# IMPORTANT:
# If you want to input a png-sequence, just insert the png's with xxxx.png starting at 0001 in ./MiddleBurySet/other-data/Beanbags/ and skip this.
# TODO: Automate this through config at the beggining of the notebook.
# If you want to input a video, then use this.

# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf /content/DAIN/input_frames
%shell mkdir -p /content/DAIN/input_frames

# TODO: Work with this to remove the alpha channel
#%shell ffmpeg -i '/content/DAIN/{filename}' -filter_complex 'split [rgb_in][alpha_in]; [rgb_in][out];' '/content/DAIN/input_frames/%05d.png'

%shell ffmpeg -i '/content/DAIN/{filename}' '/content/DAIN/input_frames/%05d.png'

png_generated_count_command_result = %shell ls /content/DAIN/input_frames | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())
print(f"{pngs_generated_count} frame PNGs generated.")

13211 frame PNGs generated.


In [0]:
# Interpolation
%shell rm -rf /content/DAIN/output_frames
%shell mkdir /content/DAIN/output_frames
%cd /content/DAIN
!python colab_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --save_which=0

/content/DAIN
revise the unique id to a random numer 51453
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/51453-Sun-Mar-08-21:42/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='./model_weights/51453-Sun-Mar-08-21:42/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/51453-Sun-Mar-08-21:42', save_which=0, seed=1, time_step=0.16666666666666666, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 5 frames
The testing model weight is: ./model_weights/best.pth
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to 

In [68]:
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -i '/content/DAIN/output_frames/%05d.png' output.mkv

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
# Copy result to drive
!cp /content/DAIN/output.mkv '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'